In [80]:
%matplotlib inline
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import pylab, mlab, pyplot
from matplotlib.lines import Line2D 
from pandas import DataFrame, Series
from IPython.display import display
from IPython.core.pylabtools import figsize, getfigs
import requests
import re
import json
from scipy.special import comb
import random

# HW3

In [4]:
def hoeffding(e,n,m):
    return 2 * m / pow(math.e, 2*n*e*e)

In [8]:
hoeffding(0.05,500,1), hoeffding(0.05,1000,1), hoeffding(0.05,1500,1), hoeffding(0.05,2000,1)

(0.16416999724779763,
 0.013475893998170937,
 0.0011061687402956677,
 9.079985952496976e-05)

In [9]:
hoeffding(0.05,500,10), hoeffding(0.05,1000,10), hoeffding(0.05,1500,10), hoeffding(0.05,2000,10)

(1.6416999724779762,
 0.13475893998170937,
 0.011061687402956676,
 0.0009079985952496975)

In [10]:
hoeffding(0.05,500,100), hoeffding(0.05,1000,100), hoeffding(0.05,1500,100), hoeffding(0.05,2000,100)

(16.41699972477976,
 1.3475893998170938,
 0.11061687402956676,
 0.009079985952496976)

# HW4

## 1.

In [5]:
def vc_bound(m, d_vc, δ):
    """
    https://mostafa-samir.github.io/ml-theory-pt2/
      m number of data points
      VC dimention of d_vc
      with a confidence of 1−δ
    """
    return math.sqrt((8*d_vc*(math.log(2*m/d_vc)+1) + 8*math.log(4/δ)) / m)

In [16]:
for m in range(400000, 500000, 20000):
    print(m, vc_bound(m, 10, 0.05))

400000 0.05045390882205938
420000 0.04933225495853472
440000 0.04828569288747159
460000 0.047306119687761265
480000 0.0463866110961582


In [163]:
# use homework's rough bound
p = lambda m: 4*((2*m)**10)*(math.e ** -(m*0.05*0.05/8))
for m in range(400000, 500000, 20000):
    print(m, p(m))

400000 221896.07655418385
420000 697.7536261453016
440000 2.144842719960257
460000 0.006458125661103977
480000 1.908095187732375e-05


## 2.

In [62]:
d_vc = 50
m_h = lambda m: (m * math.e / d_vc) ** d_vc

In [63]:
N = 10000
δ = 0.05

In [64]:
math.sqrt(math.log(4 * m_h(2*N) / δ) * 8 / N)

0.5321317536024874

In [22]:
math.sqrt(2*math.log(2*N*m_h(N))/N) + math.sqrt(2*math.log(1/δ)/N) + 1/N

0.2794575796846628

In [28]:
math.sqrt((math.log(6*m_h(2*N)/δ)+1)/N)

0.18851013741923303

In [33]:
math.sqrt((4*0.5*(1+.5) + math.log(4/δ) + d_vc*math.log(N*N*math.e/d_vc))/(2*N))

0.19784020238071998

## 3.

In [56]:
N3 = 5
ΔH = lambda m: sum([comb(m,i) for i in range(0,d_vc)])

In [65]:
math.sqrt(math.log(4 * ΔH(2*N3) / δ) * 8 / N3)

4.254597220000659

In [66]:
math.sqrt(2*math.log(2*N3*ΔH(N3))/N3) + math.sqrt(2*math.log(1/δ)/N3) + 1/N3

2.813654929686762

In [67]:
math.sqrt((math.log(6*ΔH(2*N3)/δ)+1)/N3)

1.594927180054406

In [68]:
math.sqrt((4*0.5*(1+.5) + math.log(4/δ) + d_vc*math.log(N3*N3*math.e/d_vc))/(2*N3))

1.5074703183372007

## Bias and Variance (https://mostafa-samir.github.io/ml-theory-pt3/)

In [103]:
gen_pairs = lambda n: 2 * np.random.random_sample((n,2)) - 1
gen_x = lambda n: 2 * np.random.random_sample(n) - 1
f = lambda x: math.sin(math.pi * x)
def a(x):
    return (x[0]*f(x[0]) + x[1]*f(x[1]))/(x[0]*x[0] + x[1]*x[1])

In [141]:
x_pairs = gen_pairs(100000)
a_bar = np.average(np.apply_along_axis(a, 1, x_pairs))
a_bar

1.4260244922612386

In [288]:
bias = []
for _ in range(1000):
    x_pair = gen_pairs(1)
    #print(x_pair)
    a_one_pair = np.average(np.apply_along_axis(a, 1, x_pair))
    #print(a_one_pair)
    x_pair = x_pair.flatten()
    b = (np.abs(a_one_pair*x_pair[0]-f(x_pair[0])) + np.abs(a_one_pair*x_pair[1]-f(x_pair[1])))/2
    bias.append(b)
np.average(b)

0.11762704968446221

In [147]:
e = lambda x: a_bar * x - f(x)
e = np.vectorize(e)
#np.average(e(gen_x(100000)))
np.average(e(x_pairs.flatten()))

0.00121977899666468

In [153]:
variance = lambda xp: (a(xp)*xp[0] - a_bar*xp[0])**2 + (a(xp)*xp[1] - a_bar*xp[1])**2
np.sum(np.apply_along_axis(variance, 1, x_pairs)) / x_pairs.size

0.20225986182880995

In [150]:
x_pairs.size

200000